In [1]:
#import dependencies
import pandas as pd
import geopandas as gpd
import random
from shapely.geometry import Point
!pip install pandas numpy

In [2]:
#This is the file I am reading in for my analysis. 
mental_health_csv = "./Resources/Conference and map Aishath Shahama Full PhD Study June 2022 to Dec 2023.csv"

In [5]:
#Pulling in the data here
mental_health_df = pd.read_csv(mental_health_csv)

In [7]:
mental_health_df.head()

,Religious Affliation,South Asian Heritage,Cause of Mental Health problems,Treatability,Treatments,Experience or Wittnessed stigma,Incident Description
0,Islam,Maldives,Lifestyle and genetics.,No,They can be brought under control but not cured.,Experienced,Was told to go off antidepressants because dep...
1,NaN,Maldives,Genetics and mentality,No,NaN,Experienced,Annoying in time
2,Hindhu,India,Genetics,No,It's something you are born with,Experienced,NaN
3,Islam,Maldives,"Genetics, environment and trauma",Yes,"With medication, therapy and a lot of will power",Experienced,Yes
4,Islam,Maldives,Stress and uncertainty,Yes,Psychoherapy and psychopharmaceuticals,Experienced,When saying I am going to consult a Psychologi...


In [9]:
mental_health_df.columns

Index(['Religious Affliation', 'South Asian Heritage',
       'Cause of Mental Health problems', 'Treatability', 'Treatments',
       'Experience or Wittnessed stigma', 'Incident Description'],
      dtype='object')

In [11]:
mental_health_df.count()

Religious Affliation               509
South Asian Heritage               523
Cause of Mental Health problems    494
Treatability                       523
Treatments                         407
Experience or Wittnessed stigma    521
Incident Description               192
dtype: int64

In [13]:
#What were the responses to this question? 
mental_health_df['South Asian Heritage'].unique()

array(['Maldives', 'India', 'Pakistan', 'Nepal', 'Bangladesh',
       'Afghanistan', 'Sri Lanka'], dtype=object)

In [15]:
# Dictionary to store the boundaries for each country
country_boundaries = {}

# Load the shapefile or GeoJSON for each country
country_boundaries['Afghanistan'] = gpd.read_file("./Resources/geoBoundaries-AFG-ADM0_simplified.geojson")
country_boundaries['Bangladesh']= gpd.read_file("./Resources/geoBoundaries-BGD-ADM0_simplified.geojson")
country_boundaries['India'] = gpd.read_file("./Resources/geoBoundaries-IND-ADM1_simplified.geojson")
country_boundaries['Maldives'] = gpd.read_file("./Resources/geoBoundaries-MDV-ADM0_simplified.geojson")
country_boundaries['Nepal'] = gpd.read_file("./Resources/geoBoundaries-NPL-ADM0_simplified.geojson")
country_boundaries['Pakistan'] = gpd.read_file("./Resources/geoBoundaries-PAK-ADM0_simplified.geojson")
country_boundaries['Sri Lanka'] = gpd.read_file("./Resources/geoBoundaries-LKA-ADM0_simplified.geojson")

In [17]:
# Function to generate random points within a country's polygon
def generate_random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.total_bounds
    while True:
        # Generate random latitude and longitude within the bounding box
        lat = random.uniform(miny, maxy)
        lon = random.uniform(minx, maxx)
        point = Point(lon, lat)
        # Check if the point is inside the polygon
        if polygon.contains(point).any():
            return lat, lon

In [19]:
# Apply the function row by row, generating points based on the country
def assign_random_coords(row):
    country = row['South Asian Heritage']  # Assuming your CSV has a 'Country' column
    if country in country_boundaries:
        lat, lon = generate_random_point_in_polygon(country_boundaries[country])
        return pd.Series([lat, lon])
    else:
        return pd.Series([None, None])  # Handle cases where the country isn't in the dictionary

In [21]:
# Apply the function and add latitude/longitude columns
mental_health_df[['latitude', 'longitude']] = mental_health_df.apply(assign_random_coords, axis=1)

In [22]:
# View the updated dataframe
mental_health_df.head()

,Religious Affliation,South Asian Heritage,Cause of Mental Health problems,Treatability,Treatments,Experience or Wittnessed stigma,Incident Description,latitude,longitude
0,Islam,Maldives,Lifestyle and genetics.,No,They can be brought under control but not cured.,Experienced,Was told to go off antidepressants because dep...,-0.683713,73.191997
1,NaN,Maldives,Genetics and mentality,No,NaN,Experienced,Annoying in time,-0.686968,73.133228
2,Hindhu,India,Genetics,No,It's something you are born with,Experienced,NaN,17.210676,73.369991
3,Islam,Maldives,"Genetics, environment and trauma",Yes,"With medication, therapy and a lot of will power",Experienced,Yes,3.087197,73.639211
4,Islam,Maldives,Stress and uncertainty,Yes,Psychoherapy and psychopharmaceuticals,Experienced,When saying I am going to consult a Psychologi...,0.223468,73.133905


In [29]:
# Save the updated DataFrame to a new CSV file
mental_health_df.to_csv('SouthAsianMentalHealth_with_country_data.csv', index=False)

In [31]:
# Optional: Display the updated DataFrame
print(mental_health_df.head())

  Religious Affliation South Asian Heritage   Cause of Mental Health problems  \
0                Islam             Maldives           Lifestyle and genetics.   
1                  NaN             Maldives            Genetics and mentality   
2               Hindhu                India                          Genetics   
3                Islam             Maldives  Genetics, environment and trauma   
4                Islam             Maldives            Stress and uncertainty   

  Treatability                                        Treatments  \
0           No  They can be brought under control but not cured.   
1           No                                               NaN   
2           No                  It's something you are born with   
3          Yes  With medication, therapy and a lot of will power   
4          Yes            Psychoherapy and psychopharmaceuticals   

  Experience or Wittnessed stigma  \
0                     Experienced   
1                     Experien